<a href="https://colab.research.google.com/github/Jvnc0503/DL-Lab1-Animal-Sounds/blob/main/Laboratorio_1_Animal_Sounds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

drive_path = "/content/drive/MyDrive/Animal Sounds/"
local_path = "/content/data/"

!mkdir -p {local_path}

!cp "{drive_path}train.7z" {local_path}
!cp "{drive_path}test.7z" {local_path}
!cp "{drive_path}train.csv" {local_path}

Mounted at /content/drive


In [ ]:
!apt-get install p7zip-full
!7z x /content/data/train.7z -o{local_path}
!7z x /content/data/test.7z -o{local_path}

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-8).
0 upgraded, 0 newly installed, 0 to remove and 1 not upgraded.

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/data/                        1 file, 7069498436 bytes (6742 MiB)

Extracting archive: /content/data/train.7z
--
Path = /content/data/train.7z
Type = 7z
Physical Size = 7069498436
Headers Size = 727932
Method = LZMA2:23
Solid = +
Blocks = 1

  0%      0% - train/INCT17_20191113_040000_0_3.wav                                             0% 31 - train/INCT17_20191113_040000_38_41.wav

In [ ]:
import pandas as pd

train_y = pd.read_csv(f"{local_path}train.csv")
train_y.head()

,filename,SPHSUR,BOABIS,SCIPER,DENNAH,LEPLAT,RHIICT,BOALEP,BOAFAB,PHYCUV,...,SCINAS,LEPNOT,ADEMAR,BOAALM,PHYDIS,RHIORN,LEPFLA,SCIRIZ,DENELE,SCIALT
0,INCT20955_20190909_050000_0_3.wav,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,INCT20955_20190909_050000_1_4.wav,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,INCT20955_20190909_050000_2_5.wav,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,INCT20955_20190909_050000_3_6.wav,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,INCT20955_20190909_050000_4_7.wav,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
!ls -lh /content/data/train/ | head -n 10

total 7.9G
-rw-r--r-- 1 root root 130K Apr 30  2025 INCT17_20191113_040000_0_3.wav
-rw-r--r-- 1 root root 130K Apr 30  2025 INCT17_20191113_040000_10_13.wav
-rw-r--r-- 1 root root 130K Apr 30  2025 INCT17_20191113_040000_11_14.wav
-rw-r--r-- 1 root root 130K Apr 30  2025 INCT17_20191113_040000_12_15.wav
-rw-r--r-- 1 root root 130K Apr 30  2025 INCT17_20191113_040000_13_16.wav
-rw-r--r-- 1 root root 130K Apr 30  2025 INCT17_20191113_040000_14_17.wav
-rw-r--r-- 1 root root 130K Apr 30  2025 INCT17_20191113_040000_1_4.wav
-rw-r--r-- 1 root root 130K Apr 30  2025 INCT17_20191113_040000_15_18.wav
-rw-r--r-- 1 root root 130K Apr 30  2025 INCT17_20191113_040000_16_19.wav


In [ ]:
import soundfile as sf

file_path = f"{local_path}/train/INCT17_20191113_040000_0_3.wav"
data = sf.info(file_path)

samplerate = data.samplerate

print(f"Sampling Rate exacto: {data.samplerate} Hz")
print(f"Canales: {data.channels}")
print(f"Duración: {data.duration} segundos")
print(f"Formato: {data.format_info} - {data.subtype_info}")

Sampling Rate exacto: 22050 Hz
Canales: 1
Duración: 3.0 segundos
Formato: WAV (Microsoft) - Signed 16 bit PCM


In [ ]:
import librosa
import numpy as np
import matplotlib.pyplot as plt

def generate_spectrogram(audio_path: str, image_path: str) -> None:
  y, sr = librosa.load(audio_path)
  n_fft = 2048
  hop_length = 295
  n_mels = 224

  spectrogram = librosa.feature.melspectrogram(
      y=y,
      sr=sr,
      n_fft=n_fft,
      hop_length=hop_length,
      n_mels=n_mels,
      fmin=50,
      fmax=sr/2
  )

  spectrogram_pcen = librosa.pcen(
      spectrogram * (2**31),
      time_constant=0.06,
      gain=0.8,
      bias=10,
      power=0.25,
      eps=1e-6
  )

  if spectrogram_pcen.shape[1] > 224:
    spectrogram_pcen = spectrogram_pcen[:, :224]

  if spectrogram_pcen.shape[1] < 224:
    pad_width = 224 - spectrogram_pcen.shape[1]
    spectrogram_pcen = np.pad(
        spectrogram_pcen,
         ((0,0), (0, pad_width)),
        mode='constant'
    )

  plt.imsave(
      image_path,
      spectrogram_pcen,
      cmap='magma',
      origin='lower',
      format='png'
  )

In [ ]:
import os
from pathlib import Path
from tqdm import tqdm

train_dir = Path(local_path) / "train"
test_dir = Path(local_path) / "test"
output_dir = Path(local_path) / "spectograms"

output_dir.mkdir(parents=True, exist_ok=True)

wav_files = list(train_dir.glob("*.wav")) + list(test_dir.glob("*.wav"))
print(f"Total de archivos WAV: {len(wav_files)}")

for wav_path in tqdm(wav_files, desc="Generando espectogramas"):
  try:
    save_name = wav_path.with_suffix(".png").name
    save_path = output_dir / save_name

    if save_path.exists():
      continue

    generate_spectrogram(str(wav_path), str(save_path))
  except Exception as e:
    print(f"\nError procesando {wav_path.name}: {e}")

print("¡Procesamiento completado!")

Total de archivos WAV: 93378


Generando espectogramas: 100%|██████████| 93378/93378 [2:03:18<00:00, 12.62it/s]

¡Procesamiento completado!


In [19]:
import pathlib

!du -sh {output_dir}

dir_name = Path(output_dir).name
destination_path = f"/content/drive/MyDrive/{dir_name}/"

print(f"Destination path: {destination_path}")

!cp -r "{output_dir}" "{destination_path}"
print("Copy to Drive complete")

11G	/content/data/spectograms
Destination path: /content/drive/MyDrive/spectograms/
Copy to Drive complete
/bin/bash: line 1: -head: command not found


In [21]:
!ls {destination_path} | head -n 10

INCT17_20191113_040000_0_3.png
INCT17_20191113_040000_10_13.png
INCT17_20191113_040000_11_14.png
INCT17_20191113_040000_12_15.png
INCT17_20191113_040000_13_16.png
INCT17_20191113_040000_14_17.png
INCT17_20191113_040000_1_4.png
INCT17_20191113_040000_15_18.png
INCT17_20191113_040000_16_19.png
INCT17_20191113_040000_17_20.png
